# Example EEG Feature Extraction

In [1]:
!pip install mne
!pip install numpy
!pip install pickle
!pip install mne_features

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)

[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
import mne as mne
import numpy as np
import pickle
import pandas as pd
from mne_features.feature_extraction import extract_features, FeatureExtractor

In [3]:
def load_object(fname):
    try:
        with open(fname + ".pickle", "rb") as f:
            return pickle.load(f)
    except Exception as ex:
        print("Error during unpickling object (Possibly unsupported):", ex)
        
eeg_ec_freq_bands = load_object(r"data/example_eeg/example_subj_EC_preproc")
eeg_eo_freq_bands = load_object(r"data/example_eeg/example_subj_EO_preproc")

In [4]:
def make_epochs(raw, dur, ovl):
    epochs = mne.make_fixed_length_epochs(
        raw,
        duration=dur,
        overlap=ovl,
        preload=True,
        verbose=False
    )
    return np.array(epochs.get_data(units='uV'))

In [5]:
feat_extc = {
    'only_whole_spec': {
        'pow_freq_bands',
        'wavelet_coef_energy',
    },
    'methods': {
        'std',
        'spect_slope',
        'svd_fisher_info',
        'hjorth_complexity',
        'hjorth_complexity_spect',
        'ptp_amp',
        'quantile',
        'line_length',
        'zero_crossings',
        'skewness',
        'kurtosis',
        'higuchi_fd',
        'samp_entropy',
        'app_entropy',
        'spect_entropy',
        'mean',
        'hurst_exp'
    }
}

freq_bands_of_interest = {
    'delta': [0.5, 4],
    'theta': [4, 8],
    'alpha': [8, 14],
    'beta': [14, 30],
    'whole_spec': [0.5, 30],
}

In [7]:
def get_param_for_methods(methods, fboi, fb):
    param = dict()
    if 'quantile' in methods:
        param['quantile__q'] = np.array([0.05, 0.25, 0.75, 0.95])
    if 'pow_freq_bands' in methods:
        param['pow_freq_bands__freq_bands'] = fboi
        param['pow_freq_bands__log'] = True
        param['pow_freq_bands__ratios_triu'] = False
        param['pow_freq_bands__ratios'] = None
    if 'energy_freq_bands' in methods:
        param['energy_freq_bands__freq_bands'] = fboi
    if 'spect_slope' in methods:
        param['spect_slope__fmin'] = np.array([fboi[fb][0]])
        param['spect_slope__fmax'] = np.array([fboi[fb][1]])
        param['spect_slope__with_intercept'] = True
    return param

def extract_methods(eeg_fb_dict, epoch_dur, epoch_ovl, set_methods=feat_extc, fboi=freq_bands_of_interest):
    methods_whole_spec = set_methods['only_whole_spec']
    methods = set_methods['methods']
    
    fb_epochs = dict()
    for key in eeg_fb_dict.keys():
        print(f"Extracting features in frequency band {key}")
        raw_fb = eeg_fb_dict[key]
        
        epochs = make_epochs(raw_fb.copy(), epoch_dur, epoch_ovl)
        if key == 'whole_spec':
            methods = set(methods).union(methods_whole_spec)
        
        param = get_param_for_methods(methods, fboi, key)
        
        features = extract_features(
            X=epochs,
            sfreq=raw_fb.info['sfreq'],
            selected_funcs=methods,
            funcs_params=param,
            n_jobs=5,
            ch_names=raw_fb.info['ch_names'],
            return_as_df=True
        )
        fb_epochs[key] = features
    return fb_epochs

In [15]:
print("Extracting features from eyes closed eeg")
eeg_ec_features = extract_methods(
    eeg_ec_freq_bands, 
    epoch_dur=3.9, epoch_ovl=0, 
    set_methods=feat_extc, 
    fboi=freq_bands_of_interest
)
print("")
print("Extracting features from eyes opened eeg")
eeg_eo_features = extract_methods(
    eeg_eo_freq_bands, 
    epoch_dur=1.9, epoch_ovl=0, 
    set_methods=feat_extc, 
    fboi=freq_bands_of_interest
)

Extracting features from eyes closed eeg
Extracting features in frequency band delta
Extracting features in frequency band theta
Extracting features in frequency band alpha
Extracting features in frequency band beta
Extracting features in frequency band whole_spec

Extracting features from eyes opened eeg
Extracting features in frequency band delta
Extracting features in frequency band theta
Extracting features in frequency band alpha
Extracting features in frequency band beta
Extracting features in frequency band whole_spec


In [13]:
print(eeg_ec_features['delta']['mean'])

         R1        R2        R3        R4        R5        R6        R7   
0 -0.439975  0.260699 -0.308980  0.185335  0.141282 -7.542704  0.366829  \
1  2.178752  0.142225 -0.919996 -1.317971 -2.272324  9.157061 -1.304053   
2 -1.595772 -0.130305  0.558210  1.171818  2.048781  0.816726  1.207425   
3 -0.429545 -0.051471  0.058538 -0.322971 -0.341625 -1.089214 -0.166099   
4  0.344585  0.012906  0.031625  0.266926  0.463104  0.869133  0.224787   
5 -0.114275 -0.149234  0.249515  0.445651 -0.218292 -0.326035 -0.354522   
6 -0.519357 -0.020722  0.098017 -0.460539 -0.337000 -0.142680  0.281700   
7  0.033943  0.025606 -0.472675 -0.508760  0.244248  0.223737 -0.143377   
8  0.649160  0.212927  0.127218  0.278233  0.316413  0.277050  0.071139   
9 -0.323352 -0.387368  0.498159  0.780476  0.182193  0.469174  0.146696   

         R8        R9       R10       R11       R12  
0  0.186900  0.076736  0.712260 -0.404346  0.064316  
1 -1.531724  1.365902 -0.390867 -0.049060 -0.837109  
2  1.438656 